In [111]:
import os
import re

def load_images_info(directory_path):
    images_info = []
    
    for filename in os.listdir(directory_path):
        if not filename.endswith(".png"):
            continue
        
        # Updated regex pattern to handle spaces in 'act'
        match = re.match(r"env_(\d+)_(\d+)_(\w+\s*\w*)_x_(\d+)_y_(\d+)_d_([\w_]+)\.png", filename)
        
        if match:
            env_id = int(match.group(1))
            image_id = int(match.group(2))
            act = match.group(3)   # This will now capture "pick up" and "drop off" as well
            x = int(match.group(4))
            y = int(match.group(5))
            direction = match.group(6).replace('_', '/')
            
            image_info = {
                'env_id': env_id,
                'image_id': image_id,
                'act': act,
                'x': x,
                'y': y,
                'direction': direction,
                'filename': filename
            }
            
            images_info.append(image_info)
    
    return sorted(images_info, key=lambda x: x['image_id'])

# Example usage
directory = r"../output/seed_23/train/GPT/ENV 0 steps 200 gpt 0 temp 0.7 view 7/2023-08-20 23-23-41"
images_data = load_images_info(directory)
# for image_data in images_data:
    # print(image_data)
images_data

[{'env_id': 0,
  'image_id': 0,
  'act': 'forward',
  'x': 2,
  'y': 1,
  'direction': 'Right/East',
  'filename': 'env_0_0_forward_x_2_y_1_d_Right_East.png'},
 {'env_id': 0,
  'image_id': 1,
  'act': 'left',
  'x': 2,
  'y': 2,
  'direction': 'Right/East',
  'filename': 'env_0_1_left_x_2_y_2_d_Right_East.png'},
 {'env_id': 0,
  'image_id': 2,
  'act': 'pick up',
  'x': 2,
  'y': 2,
  'direction': 'Up/North',
  'filename': 'env_0_2_pick up_x_2_y_2_d_Up_North.png'},
 {'env_id': 0,
  'image_id': 3,
  'act': 'left',
  'x': 2,
  'y': 2,
  'direction': 'Up/North',
  'filename': 'env_0_3_left_x_2_y_2_d_Up_North.png'},
 {'env_id': 0,
  'image_id': 4,
  'act': 'forward',
  'x': 2,
  'y': 2,
  'direction': 'Left/West',
  'filename': 'env_0_4_forward_x_2_y_2_d_Left_West.png'},
 {'env_id': 0,
  'image_id': 5,
  'act': 'forward',
  'x': 2,
  'y': 1,
  'direction': 'Left/West',
  'filename': 'env_0_5_forward_x_2_y_1_d_Left_West.png'},
 {'env_id': 0,
  'image_id': 6,
  'act': 'forward',
  'x': 2,
  

In [112]:
def extract_actions(images_data):
    return [image_info['act'] for image_info in images_data]

In [121]:
import re

def extract_past_actions(s: str) -> list:
    # Find the pattern for past actions using regex
    match = re.search(r'3\. Past actions (.+?)\n', s)
    if match:
        actions = [action.strip() for action in match.group(1).split(', ')]
        return actions
    else:
        return []
    
# Test the function
s = '''Follow formats in previous examples, describe about it. 
0. World ID is # 0 and your position is (4, 1), facing Down/South. 
1. World Map 
[[- - - - - - - - - - -]
 [5Wall 0Empty 0Empty 0Empty 0Empty - - - - - -]
 [5Wall 0Empty 1Key 1Ball 0Empty - - - - - -]
 [5Wall 0Empty 0Empty 0Empty 0Empty - - - - - -]
 [5Wall Yourself 0Empty 0Empty 0Empty - - - - - -]
 [5Wall 5Wall 5Wall 5Wall 5Wall - - - - - -]] 
2. Inventory You are holding nothing 
3. Past actions pick up, forward, forward, forward, left, forward, forward, toggle, forward, drop off
4. Gained experience Comparing my new experience with my previous experience, I have confirmed that my position remains unchanged at (4, 1) facing Down/South. The empty space (0Empty) in front of me continues to offer a potential path for further exploration. The key (1Key) remains in my inventory, indicating that I have successfully acquired it. Armed with this information, I can plan my next actions to continue exploring the world, potentially unlocking locked objects and uncovering new discoveries. 
Limit your description to be less than 50 tokens in total.'''

print(extract_past_actions(s))


['pick up', 'forward', 'forward', 'forward', 'left', 'forward', 'forward', 'toggle', 'forward', 'drop off']


In [129]:
def combine_actions(old_action: list, new_action: list, N: int) -> list:
    # Calculate how many actions need to be trimmed from the old_action
    trim_length = len(old_action) + len(new_action) - N

    # If trim_length is negative or zero, no need to trim
    if trim_length < 0:
        trim_length = 0

    # Trim the old_action and append the new_action
    combined = old_action[trim_length:] + new_action

    return combined

# Test the function
new_action = ['toggle', 'forward', 'right', 'forward']
old_action = ['pick up', 'forward', 'forward', 'forward', 'left', 'forward', 'forward', 'toggle', 'forward', 'drop off']
N = 5

print(combine_actions(old_action, new_action, N)) # This will append new_action to the end of old_action since N is larger


['drop off', 'toggle', 'forward', 'right', 'forward']


In [123]:
import ast

def string_to_list(s: str) -> list:
    return ast.literal_eval(s)

# Test the function
s = "['toggle', 'forward', 'right', 'forward', 'pick up']"
print(string_to_list(s))

['toggle', 'forward', 'right', 'forward', 'pick up']


In [113]:
# Using the function
actions_list = extract_actions(images_data)
print(actions_list)

['forward', 'left', 'pick up', 'left', 'forward', 'forward', 'forward', 'forward', 'forward', 'right', 'toggle', 'left', 'forward', 'toggle', 'right', 'forward', 'toggle', 'right', 'toggle', 'forward', 'left', 'forward', 'toggle', 'forward', 'pick up', 'right', 'toggle', 'forward', 'right', 'forward', 'left', 'right', 'toggle', 'forward', 'pick up', 'right', 'toggle', 'right', 'forward', 'pick up', 'left', 'right', 'forward', 'pick up', 'forward', 'forward', 'toggle', 'forward', 'right', 'forward', 'pick up', 'right', 'forward', 'toggle', 'forward', 'pick up', 'forward', 'toggle', 'right', 'right', 'toggle', 'forward', 'toggle', 'pick up', 'drop off', 'forward', 'toggle', 'forward', 'right', 'forward', 'forward', 'toggle', 'pick up', 'forward', 'toggle', 'forward', 'pick up', 'forward', 'forward', 'toggle', 'pick up', 'left', 'forward', 'toggle', 'left', 'forward', 'toggle', 'pick up', 'forward', 'toggle', 'forward', 'toggle', 'toggle', 'forward', 'pick up', 'forward', 'right', 'forwar

In [115]:
len(actions_list)

503

In [20]:
env_id = images_data[0]["env_id"]
length_table_n = f"length_table_env_{env_id}.csv"
metrics_table_n = f"metrics_table_env_{env_id}.csv"
rec_table_n = f"rec_table_env_{env_id}.csv"
scn_table_n = f"scn_table_env_{env_id}.csv"
world_map_table_n = f"world_map_table_env_{env_id}.csv"

In [21]:
import pandas as pd

In [23]:
length_table = pd.read_csv(length_table_n)
metrics_table = pd.read_csv(metrics_table_n)
rec_table = pd.read_csv(rec_table_n)
scn_table = pd.read_csv(scn_table_n)
world_map_table = pd.read_csv(world_map_table_n)

In [81]:
print(world_map_table.iloc[-1]["world_map_obj"])

[[- - - - - - - - - - -]
 [2 1 1 1 - - - - - - -]
 [2 1 5 6 - - - - - - -]
 [2 1 1 1 - - - - - - -]
 [2 Yourself 1 1 1 - - - - - -]
 [2 2 2 2 2 - - - - - -]]


In [30]:
len(rec_table)

504

In [37]:
length_table.iloc[-1]

Unnamed: 0        199
sum              1753
desc_sys           25
desc_user_0       665
desc_assis         64
desc_user_1       309
desc               50
reason_user_0     154
reason            100
n_exp_user_0      214
n_exp              42
s_exp_user_0       33
s_sxp              97
Name: 199, dtype: int64

In [38]:
metrics_table.iloc[-1]

Unnamed: 0    502.000000
env_view       54.545455
env_step       24.242424
env_memo       33.333333
obj_view       54.545455
toggle         45.454545
pick up        45.454545
drop off       18.181818
Name: 502, dtype: float64

In [40]:
rec_table.iloc[-1]["env_view"]

'[[104 138 129 131 139 115   0   0   0   0   0]\n [150 192 172 171 202 172   0   0   0   0   0]\n [143 177 157 156 187 161   0   0   0   0   0]\n [147 181 161 160 191 165   0   0   0   0   0]\n [169 217 197 196 227 187   0   0   0   0   0]\n [150 191 166 164 192 157   0   0   0   0   0]]'

In [48]:
import numpy as np

data_string = '[[104 138 129 131 139 115   0   0   0   0   0]\n [150 192 172 171 202 172   0   0   0   0   0]\n [143 177 157 156 187 161   0   0   0   0   0]\n [147 181 161 160 191 165   0   0   0   0   0]\n [169 217 197 196 227 187   0   0   0   0   0]\n [150 191 166 164 192 157   0   0   0   0   0]]'

# Remove newline characters and split the string into individual numbers
cleaned_string = data_string.replace('\n', '').replace('[', '').replace(']', '')
numbers = cleaned_string.split()

# Convert the numbers into integers
int_numbers = [int(num) for num in numbers]

# Reshape the list of numbers into a 2D array
num_rows = 6
num_columns = 11
numpy_array = np.array(int_numbers).reshape(num_rows, num_columns)

# Convert the int array to float and create env_view
env_view = numpy_array.astype(np.float64)

print(numpy_array)
print(env_view)

[[104 138 129 131 139 115   0   0   0   0   0]
 [150 192 172 171 202 172   0   0   0   0   0]
 [143 177 157 156 187 161   0   0   0   0   0]
 [147 181 161 160 191 165   0   0   0   0   0]
 [169 217 197 196 227 187   0   0   0   0   0]
 [150 191 166 164 192 157   0   0   0   0   0]]
[[104. 138. 129. 131. 139. 115.   0.   0.   0.   0.   0.]
 [150. 192. 172. 171. 202. 172.   0.   0.   0.   0.   0.]
 [143. 177. 157. 156. 187. 161.   0.   0.   0.   0.   0.]
 [147. 181. 161. 160. 191. 165.   0.   0.   0.   0.   0.]
 [169. 217. 197. 196. 227. 187.   0.   0.   0.   0.   0.]
 [150. 191. 166. 164. 192. 157.   0.   0.   0.   0.   0.]]


In [80]:
def string_to_object_array(data):
    # Split the data string by newlines
    lines = data.split('\n')
    
    # For each line, strip whitespace, remove outer square brackets, 
    # split by spaces, and keep the value as is.
    rows = [line.strip().replace('[', '').replace(']', '').split() for line in lines]
    
    # Convert to a numpy array with dtype 'object'
    array = np.array(rows, dtype=object)
    
    return array



data_str = world_map_table.iloc[-1]["world_map_obj"]

result = string_to_object_array(data_str)
print(result)

print(result.shape)

result

[['-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-']
 ['2' '1' '1' '1' '-' '-' '-' '-' '-' '-' '-']
 ['2' '1' '5' '6' '-' '-' '-' '-' '-' '-' '-']
 ['2' '1' '1' '1' '-' '-' '-' '-' '-' '-' '-']
 ['2' 'Yourself' '1' '1' '1' '-' '-' '-' '-' '-' '-']
 ['2' '2' '2' '2' '2' '-' '-' '-' '-' '-' '-']]
(6, 11)


array([['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'],
       ['2', '1', '1', '1', '-', '-', '-', '-', '-', '-', '-'],
       ['2', '1', '5', '6', '-', '-', '-', '-', '-', '-', '-'],
       ['2', '1', '1', '1', '-', '-', '-', '-', '-', '-', '-'],
       ['2', 'Yourself', '1', '1', '1', '-', '-', '-', '-', '-', '-'],
       ['2', '2', '2', '2', '2', '-', '-', '-', '-', '-', '-']],
      dtype=object)

In [83]:
for row in result:
    print(row[0])

-
2
2
2
2
2


In [88]:
print(scn_table.iloc[-1]["s_exp"])

Comparing my new experience with my previous experience, I have confirmed that my position remains unchanged at (4, 1) facing Left/West. The empty space (0Empty) in front of me continues to offer a potential path for further exploration. The key (1Key) remains in my inventory, indicating that I have successfully acquired it. Armed with this information, I can plan my next actions to continue exploring the world, potentially unlocking locked objects and uncovering new discoveries.


In [86]:
import numpy as np

def string_to_numpy_array(data):
    # Remove outer brackets and split by spaces and newlines.
    numbers = [float(num) if '\n' in data else int(num) for num in data.replace('[', '').replace(']', '').split() if num.strip()]

    # Create numpy array
    array = np.array(numbers, dtype=np.float64 if '\n' in data else int)

    # Reshape if the original string is 2D
    if '\n' in data:
        # Get the number of rows by counting newline characters and adding 1.
        rows = data.count('\n') + 1
        # Reshape the array
        array = array.reshape(rows, -1)

    return array

data_str = rec_table.iloc[-1]["env_view"]

result = string_to_numpy_array(data_str)
print(result)

print(result.shape[0], result.shape[1])

result

[[104. 138. 129. 131. 139. 115.   0.   0.   0.   0.   0.]
 [150. 192. 172. 171. 202. 172.   0.   0.   0.   0.   0.]
 [143. 177. 157. 156. 187. 161.   0.   0.   0.   0.   0.]
 [147. 181. 161. 160. 191. 165.   0.   0.   0.   0.   0.]
 [169. 217. 197. 196. 227. 187.   0.   0.   0.   0.   0.]
 [150. 191. 166. 164. 192. 157.   0.   0.   0.   0.   0.]]
6 11


array([[104., 138., 129., 131., 139., 115.,   0.,   0.,   0.,   0.,   0.],
       [150., 192., 172., 171., 202., 172.,   0.,   0.,   0.,   0.,   0.],
       [143., 177., 157., 156., 187., 161.,   0.,   0.,   0.,   0.,   0.],
       [147., 181., 161., 160., 191., 165.,   0.,   0.,   0.,   0.,   0.],
       [169., 217., 197., 196., 227., 187.,   0.,   0.,   0.,   0.,   0.],
       [150., 191., 166., 164., 192., 157.,   0.,   0.,   0.,   0.,   0.]])

In [69]:
np.array([0 for i in range(11)])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [49]:
env_view

array([[104., 138., 129., 131., 139., 115.,   0.,   0.,   0.,   0.,   0.],
       [150., 192., 172., 171., 202., 172.,   0.,   0.,   0.,   0.,   0.],
       [143., 177., 157., 156., 187., 161.,   0.,   0.,   0.,   0.,   0.],
       [147., 181., 161., 160., 191., 165.,   0.,   0.,   0.,   0.,   0.],
       [169., 217., 197., 196., 227., 187.,   0.,   0.,   0.,   0.,   0.],
       [150., 191., 166., 164., 192., 157.,   0.,   0.,   0.,   0.,   0.]])

In [50]:
a = np.zeros((6,8))
a

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [1]:
import json
# Load the utilities JSON
def load_json(address):
    with open(address, 'r', encoding='utf-8') as f:
        return json.load(f)

In [4]:
import csv

In [6]:
import pandas as pd

In [7]:
rec_table = pd.read_csv(r"/home/hz271/Research/General Robotics Lab 通用机器人实验室/LLM_As_Agent/output/seed_23/train/GPT/ENV 0 steps 100 gpt 0 temp 0.7 view 3/2023-08-20 14-08-58/rec_table_env_0.csv")

In [13]:
for i in range(63):
    print(rec_table["env_memo"][i])
    print()
    print(rec_table["env_step"][i])
    print()

[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1  4  4  4 -1 -1 -1 -1 -1 -1 -1]
 [-1  4  4  4 -1 -1 -1 -1 -1 -1 -1]
 [-1  4  4  4 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]

[[0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]]

[[-2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2]
 [-2  3  8  8  3 -2 -2 -2 -2 -2 -2]
 [-2  3  8  8  3 -2 -2 -2 -2 -2 -2]
 [-2  3  8  8  3 -2 -2 -2 -2 -2 -2]
 [-2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2]
 [-2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2]]

[[0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]]

[[-3  2  2  2 -3 -3 -3 -3 -3 -3 -3]
 [-3  7 10 10  2 -3 -3 -3 -3 -3 -3]
 [-3  7 10 10  2 -3 -3 -3 -3 -3 -3]
 [-3  2  7  7  2 -3 -3 -3 -3 -3 -3]
 [-3 -3 -3 -3 -3 -3 -3 -3 -3 -3 -3]
 [-3 -3 -3 -3 -3 -3 -3 -3 -3 -3 -3]]

[[0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0

In [ ]:
rec_table = load_json(r"/home/hz271/Research/General Robotics Lab 通用机器人实验室/LLM_As_Agent/output/seed_23/train/GPT/ENV 0 steps 100 gpt 0 temp 0.7 view 3/2023-08-20 14-08-58/rec_table_env_0.csv")

In [11]:
train_msg = load_json(r"/home/hz271/Research/General Robotics Lab 通用机器人实验室/LLM_As_Agent/data/input/msg/train_msg.json")

In [12]:
print(train_msg["desc_user_0"])

This is a text-based small 2D world, you mission is to explore it as much as you can, understand surrounding's object's properties and summarize useful experience. Observations are composed of: 
0. World ID, your position and your facing direction. 
1. World Map: A 2D matrix that records yourself and surrounding objects, where each object is encoded by this rule: If 5 is grey, then 5Wall is grey wall. 1 is green, 1Door+ will mean a green door but locked because + represents locked. Yourself is the position of you in the world map, with coordinate being (X, Y) and your facing direction (Right/East, Up/North, Left/West or Down/South). You cannot see objects behind you or blocked behind. The world map will be updated as you explore around. You will forget about positions that you haven't explored for a while. Places you have forgotten or have not viewed yet will all be marked as unseen (-).
 An example:
[[- 5Wall 5Wall 5Wall 5Wall 5Wall - - - - -] 
 [- 0Empty 0Empty 0Empty 1Ball 1Door+ - 

In [15]:
import tiktoken

In [16]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [17]:
a = """This is a text-based small 2D world, you mission is to explore it as much as you can, understand surrounding's object's properties and summarize useful experience. Observations are composed of: 
0. World ID, your position and your facing direction. 
1. World Map: A 2D matrix that records yourself and surrounding objects, where each object is encoded by this rule: If 5 is grey, then 5Wall is grey wall. 1 is green, 1Door+ will mean a green door but locked because + represents locked. Yourself is the position of you in the world map, with coordinate being (X, Y) and your facing direction (Right/East, Up/North, Left/West or Down/South). You cannot see objects behind you or blocked behind. The world map will be updated as you explore around. You will forget about positions that you haven't explored for a while. Places you have forgotten or have not viewed yet will all be marked as unseen (-).
 An example:
[[- 5Wall 5Wall 5Wall 5Wall 5Wall - - - - -] 
 [- 0Empty 0Empty 0Empty 1Ball 1Door+ - - - - -] 
 [- Yourself 0Empty 0Empty 1Key 5Wall - - - - -] 
 [- 0Empty 0Empty 0Empty 0Empty 5Wall - - - - -] 
 [- 0Empty 0Empty 0Empty 0Empty 5Wall - - - - -] 
 [- 5Wall 5Wall 5Wall 5Wall 5Wall - - - - -]] 
 The numbers represents color, where black is 0, green is 1, blue is 2, purple is 3, yellow is 4 and grey is 5. Symbols represent status, nothing is open, + is locked, * is closed. For example 5Door* is a green closed door. Your position (X, Y) and facing direction. For example in this world map your position is (2,1) and direction is East/Right.
Interpretation can be:
a. You are in a room with walls that are grey (5Wall) on the top, bottom, and right side.
b. Three spaces in front of you, to your left/west, is a green ball (1Ball).
c. Four spaces in front of you, one space to your left, is a green locked (+) door (1Door+). 
 2. Inventory. You can only hold one item with you. For example you are holding a key, then you can't pick up anything unless you dropped off your key to an droppable object. 
 3. Past actions. You can remember your latest few past actions, try using diverse actions as they help to explore the world. An example:  Your past actions are forward, forward 
 Interpretation: You have tried forward two times.
4. Gained experience:
You can refer to your gained past experiences to guide you to explore the world. 
 All these examples are just for your reference, your real observation will be provided to you later with a similar format."""

In [18]:
len(encoding.encode(a))

665

In [22]:
len(encoding.encode("what"))

1

In [9]:
import os
os.curdir

'.'

In [10]:
import openai
openai.api_key = open("../utils/api/API_KEY", "r").read()

In [93]:
def choose_act(action):
    # Check if param is a list
    if isinstance(action, list):
        # Check if the list has more than 1 element
        if len(action) > 1:
            return action
        else:
            # If it's a list of length 1, extract the first element and try to convert it to int
            try:
                return int(action[0])
            except (ValueError, TypeError):
                raise ValueError("Single element in the list or the parameter itself could not be converted to an integer.")
    else:
        # If it's not a list, try to convert it to int
        try:
            return int(action)
        except (ValueError, TypeError):
            raise ValueError("The parameter could not be converted to an integer.")

In [94]:
msg = [{"role": "user", "content": "convert 1, 2 into a list of integers by calling function \"choose_act\" with list of integers as parameter"}]

In [101]:
msg = [{"role": "user", "content": "Choose an action or a mutiple step actions plan from the following to exlore the environment as much as you can: \n(0) left: turn your view to the left object\n(1) right: turn your view to the right object\n(2) toggle: toggle the object in front of you\n(3) forward: move yourself to the front object if it's empty or opened\n(4) pick up: pick up the front object if it can be picked up\n(5) drop off: drop off the current inventory if front is empty\n\nAnd explain why.\n\nLimit your reason to be less than 100 tokens.\n\nUse format like: (a). Action #1 (b). Action #2 etc."}]
msg.append({"role": "assistant", "content": "I choose to do move forward four times"})
msg.append({"role": "user", "content": "Convert above reason of choices into a list of integers by calling function \"choose_act\" with a list of integers as parameter"})

In [102]:

fuc_msg = "Function to convert the reason of choices into a list of integers"
fuc = [
    {
        "name": "choose_act",
        "description":fuc_msg,
        "parameters":{
            "type":"object",
            "properties":{
                "action":
                {
                    "type":"integer",
                    "description":"list of integers corresponding to the action",
                }  
            },
            "required":["action"]
        }
    }]

In [103]:
import json
rsp = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = msg,
    functions = fuc,
    function_call = "auto",
    temperature = 0.1
)
rsp_msg = rsp["choices"][0]["message"]
if rsp_msg.get("function_call"):
    fuc_l = {
        "choose_act": choose_act,
    }
    fuc_n = rsp_msg["function_call"]["name"]
    if fuc_n == "choose_act":
        fuc_c = fuc_l[fuc_n]
        fuc_args = json.loads(rsp_msg["function_call"]["arguments"])
        act = fuc_c(
            action = fuc_args.get("action")
        )
        

In [104]:
type(act)

list

In [105]:
act

[3, 3, 3, 3]

In [106]:
rsp["choices"]

[<OpenAIObject at 0x7f504f2735b0> JSON: {
   "index": 0,
   "message": {
     "role": "assistant",
     "content": null,
     "function_call": {
       "name": "choose_act",
       "arguments": "{\n  \"action\": [3, 3, 3, 3]\n}"
     }
   },
   "finish_reason": "function_call"
 }]

In [107]:
print(rsp_msg)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "choose_act",
    "arguments": "{\n  \"action\": [3, 3, 3, 3]\n}"
  }
}


In [2]:
import sys
sys.path.append("..") 

In [13]:
import json
# Load the utilities JSON
def load_json(address):
    with open(address, 'r', encoding='utf-8') as f:
        return json.load(f)
    
def get_pos_m(env_pos):
    pos_m = {}
    for env in env_pos.strip().split("\n"):
        env_id, x, y, direction = env.strip().split(', ')
        pos_m[env_id] = (int(x), int(y), direction)
    return pos_m

env_ids = load_json(r"../data/input/env/env_ids.json")


import gymnasium as gym
from classes.Minigrid.minigrid.core.actions import Actions
from classes.Minigrid.minigrid.minigrid_env import MiniGridEnv

def start_env(env_id):
    env: MiniGridEnv = gym.make(
        id = env_ids[str(env_id)],
        render_mode = "rgb_array",
        agent_view_size = 7,
        screen_size = 640
    )
    return env

def reset_env(env, seed):
    return env.reset(seed=seed)

In [15]:
env = start_env(0)

In [17]:
print(env.agent_pos)

None


In [2]:
def extract_inventory(s: str) -> int:
    # Correspondence relationship
    inventory_map = {
        "nothing": "0",
        "Empty": "1",
        "Wall": "2",
        "Floor": "3",
        "Door": "4",
        "Key": "5",
        "Ball": "6",
        "Box": "7",
        "Portal": "8",
        "Lava": "9"
    }

    # Extract the inventory line from the string
    inventory_line = [line for line in s.split('\n') if "Inventory" in line][0]

    # Extract the actual inventory item
    inventory_item = inventory_line.split("You are holding ")[1].strip()

    # Convert the item to its integer representation
    return int(inventory_map[inventory_item])

# Example usage
s = """Follow formats in previous examples, describe about it. 
0. World ID is # 0 and your position is (4, 1), facing Down/South. 
1. World Map 
[[- - - - - - - - - - -]
 [5Wall 0Empty 0Empty 0Empty 0Empty - - - - - -]
 [5Wall 0Empty 1Key 1Ball 0Empty - - - - - -]
 [5Wall 0Empty 0Empty 0Empty 0Empty - - - - - -]
 [5Wall Yourself 0Empty 0Empty 0Empty - - - - - -]
 [5Wall 5Wall 5Wall 5Wall 5Wall - - - - - -]] 
2. Inventory You are holding Key
3. Past actions pick up, forward, forward, forward, left, forward, forward, toggle, forward 
4. Gained experience Comparing my new experience with my previous experience, I have confirmed that my position remains unchanged at (4, 1) facing Down/South. The empty space (0Empty) in front of me continues to offer a potential path for further exploration. The key (1Key) remains in my inventory, indicating that I have successfully acquired it. Armed with this information, I can plan my next actions to continue exploring the world, potentially unlocking locked objects and uncovering new discoveries. 
Limit your description to be less than 50 tokens in total."""

print(extract_inventory(s))  # Expected output: 0


5


In [18]:
import os
import pandas as pd
def load_tables(load_address, env_id):
    rec_table_n = f"rec_table_env_{env_id}.csv"
    scn_table_n = f"scn_table_env_{env_id}.csv"
    world_map_table_n = f"world_map_table_env_{env_id}.csv"
    length_table_n = f"length_table_env_{env_id}.csv"
    metrics_table_n = f"metrics_table_env_{env_id}.csv"
    rec_table = pd.read_csv(os.path.join(load_address, rec_table_n))
    scn_table = pd.read_csv(os.path.join(load_address, scn_table_n))
    world_map_table = pd.read_csv(os.path.join(load_address, world_map_table_n))
    length_table = pd.read_csv(os.path.join(load_address, length_table_n))
    metrics_table = pd.read_csv(os.path.join(load_address, metrics_table_n))
    return rec_table, scn_table, world_map_table, length_table, metrics_table

rec_table, scn_table, world_map_table, length_table, metrics_table = load_tables(r"C:\Users\holle\OneDrive - Duke University\Research\LLM_As_Agent\2023-08-20 13-55-25", 0)


In [19]:
def load_images_info(directory_path):
    images_info = []
    
    for filename in os.listdir(directory_path):
        if not filename.endswith(".png"):
            continue
        
        match = re.match(r"env_(\d+)_(\d+)_(\w+)_x_(\d+)_y_(\d+)_d_([\w_]+)\.png", filename)
        if match:
            env_id = int(match.group(1))
            image_id = int(match.group(2))
            act = match.group(3)
            x = int(match.group(4))
            y = int(match.group(5))
            direction = match.group(6).replace('_', '/')
            
            image_info = {
                'env_id': env_id,
                'image_id': image_id,
                'act': act,
                'x': x,
                'y': y,
                'direction': direction,
                'filename': filename
            }
            
            images_info.append(image_info)
    
    return sorted(images_info, key=lambda x: x['image_id'])

In [20]:
import re
images_data = load_images_info(r"C:\Users\holle\OneDrive - Duke University\Research\LLM_As_Agent\2023-08-20 13-55-25")

In [24]:
for image_data in images_data:
    env_id = image_data['env_id']
    image_id = image_data['image_id']
    act = image_data['act']
    x = image_data['x']
    y = image_data['y']
    direction = image_data['direction']
    filename = image_data['filename']
    print(metrics_table.iloc[image_id])

Unnamed: 0     0.000000
env_view      13.636364
env_step       1.515152
env_memo      13.636364
obj_view       9.090909
toggle         0.000000
pick up        0.000000
drop off       0.000000
Name: 0, dtype: float64
Unnamed: 0     1.000000
env_view      18.181818
env_step       3.030303
env_memo      18.181818
obj_view      27.272727
toggle         0.000000
pick up        0.000000
drop off       0.000000
Name: 1, dtype: float64
Unnamed: 0     2.000000
env_view      22.727273
env_step       4.545455
env_memo      22.727273
obj_view      45.454545
toggle         0.000000
pick up        0.000000
drop off       0.000000
Name: 2, dtype: float64
Unnamed: 0     3.000000
env_view      27.272727
env_step       4.545455
env_memo      27.272727
obj_view      45.454545
toggle         0.000000
pick up        0.000000
drop off       0.000000
Name: 3, dtype: float64
Unnamed: 0     4.000000
env_view      31.818182
env_step       4.545455
env_memo      18.181818
obj_view      45.454545
toggle         0

In [30]:
import pandas as pd

def concatenate_csvs(old_file, new_file, combined_file):
    # Read the old and new CSV files into pandas DataFrames
    old_df = pd.read_csv(old_file)
    new_df = pd.read_csv(new_file)
    
    # Reset the index of the new DataFrame to start where the old one left off
    new_df.index = range(old_df.index[-1] + 1, old_df.index[-1] + 1 + len(new_df))
    
    # Concatenate the two DataFrames
    combined_df = pd.concat([old_df, new_df], axis=0)
    
    # Save the combined DataFrame to the combined_file
    combined_df.to_csv(combined_file, index=True, index_label='index')

# Use the function
concatenate_csvs('rec_table_env_0.csv', 'retrain/rec_table_env_0.csv', 'rec_table_combined.csv')
